In [1]:
# importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels

import os
import random

In [2]:
import sklearn
import tensorflow 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

In [4]:
os.environ['PYTHONHASHSEED'] = '42'  

random.seed(42)          
np.random.seed(42)       
tensorflow.random.set_seed(42)  

In [5]:
# load the data into dataframe
df=pd.read_csv('feature_engineered_data.csv')
print(df.head())

   current_value  lights         T1       RH_1         T2       RH_2  \
0          430.0      30  20.133333  48.000000  19.566667  44.400000   
1          250.0      30  20.260000  52.726667  19.730000  45.100000   
2          100.0      10  20.426667  55.893333  19.856667  45.833333   
3          100.0      10  20.566667  53.893333  20.033333  46.756667   
4           90.0      10  20.730000  52.660000  20.166667  47.223333   

          T3       RH_3         T4       RH_4  ...  is_weekend    nsm  lag_1  \
0  19.890000  44.900000  19.000000  46.363333  ...           0  68400  576.6   
1  19.890000  45.493333  19.000000  47.223333  ...           0  69000  430.0   
2  20.033333  47.526667  19.000000  48.696667  ...           0  69600  250.0   
3  20.100000  48.466667  19.000000  48.490000  ...           0  70200  100.0   
4  20.200000  48.530000  18.926667  48.156667  ...           0  70800  100.0   

   lag_2  lag_6  lag_12  hour_sin  hour_cos  rolling_mean_12  rolling_std_12  
0  230.

In [6]:
import pickle

# Load the selected features for 10-minute forecasting
with open("selected_features_60min.pkl", "rb") as f:
    selected_features = pickle.load(f)

print("Selected Features:", selected_features)

Selected Features: ['current_value', 'T3', 'RH_5', 'RH_8', 'T_out', 'Press_mm_hg', 'nsm', 'hour_cos', 'rolling_mean_12', 'rolling_std_12']


In [7]:
df['target_60min']= df['current_value'].shift(-6)

In [8]:
df = df.dropna().reset_index(drop=True)

In [9]:
df = df[selected_features + ['target_60min']]

In [10]:
train_size = int(0.8 * len(df))
df_train, df_test = df[:train_size], df[train_size:]

In [11]:
scaler = MinMaxScaler()
df_train_scaled = pd.DataFrame(scaler.fit_transform(df_train), columns=df.columns, index=df_train.index)
df_test_scaled = pd.DataFrame(scaler.transform(df_test), columns=df.columns, index=df_test.index)


In [12]:
def create_sequences(data, target_column, window_size):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data.iloc[i-window_size:i].values)
        y.append(data.iloc[i][target_column])
    return np.array(X), np.array(y)

window_size = 24  # use previous 24 time steps
#X, y = create_sequences(scaled_df, target_column='target_10min', window_size=window_size)
X_train, y_train = create_sequences(df_train_scaled, target_column='target_60min', window_size=window_size)
X_test, y_test = create_sequences(df_test_scaled, target_column='target_60min', window_size=window_size)

In [13]:
val_size = int(len(X_train) * 0.125)

X_train_final = X_train[:-val_size]
y_train_final = y_train[:-val_size]

X_val = X_train[-val_size:]
y_val = y_train[-val_size:]


In [14]:
# Invert scaling for target only
target_scaler = MinMaxScaler()
target_scaler.fit(df[['target_60min']])  # fit only on original (unscaled) appliances column

MinMaxScaler()

In [15]:
def evaluate(model,y_pred,y_test):
    y_test_actual = target_scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred_actual = target_scaler.inverse_transform(y_pred)

    r2 = r2_score(y_test_actual, y_pred_actual)
    mae = mean_absolute_error(y_test_actual, y_pred_actual)
    rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))

    print(f"Model Name:{model}, R²: {r2:.4f}, MAE: {mae:.2f}, RMSE: {rmse:.2f}")

In [16]:
from keras.models import Sequential
from keras.layers import Dense, LSTM,GRU
from keras_tuner.tuners import BayesianOptimization
from keras.callbacks import EarlyStopping
from tcn import TCN

In [17]:
def build_tcn_lstm_model(hp):
    model = Sequential()

    # Get dilation depth (e.g., 3 → [1, 2, 4], 4 → [1, 2, 4, 8])
    dilation_depth = hp.Int('dilation_depth', 2, 4)  # depth of dilations
    dilations = [2**i for i in range(dilation_depth)]

    model.add(TCN(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        nb_filters=hp.Int('nb_filters', 32, 128, step=32),
        kernel_size=hp.Choice('kernel_size', [2, 3, 5]),
        dilations=dilations,
        return_sequences=True,
        dropout_rate=hp.Float('tcn_dropout', 0.1, 0.5, step=0.1)
    ))

    model.add(LSTM(units=hp.Int('lstm_units', 32, 128, step=32)))

    model.add(Dense(1))

    model.compile(
        optimizer=tensorflow.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mse',
        metrics=['mae']
    )

    return model



In [18]:
tuner = BayesianOptimization(
    build_tcn_lstm_model,
    objective='val_mae',
    max_trials=15,
    directory='tcn_lstm_tuning60',
    project_name='bayes_tcn_lstm'
)


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\tcn\tcn.py:268: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


In [19]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

tuner.search(
    X_train_final, y_train_final,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1,
    shuffle=False
)


Trial 15 Complete [00h 01m 46s]
val_mae: 0.0525430366396904

Best val_mae So Far: 0.043445736169815063
Total elapsed time: 00h 35m 17s


In [20]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best hyperparameters:")
for param in best_hp.values:
    print(f"{param}: {best_hp.get(param)}")


Best hyperparameters:
dilation_depth: 3
nb_filters: 32
kernel_size: 3
tcn_dropout: 0.4
lstm_units: 64
learning_rate: 0.001


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 40 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [21]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

y_pred_tcn_lstm = best_model.predict(X_test)

evaluate("TCN-LSTM",y_pred_tcn_lstm,y_test)


123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Model Name:TCN-LSTM, R²: 0.5704, MAE: 24.89, RMSE: 55.59


# TCN-GRU

In [22]:
def build_tcn_gru_model(hp):
    model = Sequential()

    # Get dilation depth (e.g., 3 → [1, 2, 4], 4 → [1, 2, 4, 8])
    dilation_depth = hp.Int('dilation_depth', 2, 4)  # depth of dilations
    dilations = [2**i for i in range(dilation_depth)]

    model.add(TCN(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        nb_filters=hp.Int('nb_filters', 32, 128, step=32),
        kernel_size=hp.Choice('kernel_size', [2, 3, 5]),
        dilations=dilations,
        return_sequences=True,
        dropout_rate=hp.Float('tcn_dropout', 0.1, 0.5, step=0.1)
    ))

    model.add(GRU(units=hp.Int('lstm_units', 32, 128, step=32)))

    model.add(Dense(1))

    model.compile(
        optimizer=tensorflow.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mse',
        metrics=['mae']
    )

    return model


In [23]:
tuner = BayesianOptimization(
    build_tcn_gru_model,
    objective='val_mae',
    max_trials=15,
    directory='tcn_gru_tuning60',
    project_name='bayes_tcn_gru'
)


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\tcn\tcn.py:268: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


In [24]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

tuner.search(
    X_train_final, y_train_final,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1,
    shuffle=False
)

Trial 15 Complete [00h 01m 00s]
val_mae: 0.1209360659122467

Best val_mae So Far: 0.0441228523850441
Total elapsed time: 00h 32m 54s


In [25]:
best_model_tcn_gru = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best hyperparameters:")
for param in best_hp.values:
    print(f"{param}: {best_hp.get(param)}")

Best hyperparameters:
dilation_depth: 4
nb_filters: 64
kernel_size: 5
tcn_dropout: 0.5
lstm_units: 128
learning_rate: 0.001


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 48 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [26]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

y_pred_tcn_gru = best_model_tcn_gru.predict(X_test)

evaluate("TCN-GRU",y_pred_tcn_gru,y_test)


123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step
Model Name:TCN-GRU, R²: 0.5683, MAE: 25.89, RMSE: 55.72
